## Rag From Scratch: Query Transformations

Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.

![](/Users/dhavalantala/Desktop/langchain_/Photo/query_transformation.png)

## Set Environment Vars and API Keys

In [37]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [38]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

## Part 5: Multi Query

Flow: 

![](/Users/dhavalantala/Desktop/langchain_/Photo/multi-query.png)


Docs:

* [https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever)

### Index

In [39]:
#### INDEXING ####

import bs4 
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS 
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

loader = WebBaseLoader(
    web_path = ("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

blog_docs = loader.load()

#### Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300,
    chunk_overlap = 100
)

#### Make Splits
splits = text_splitter.split_documents(blog_docs)

#### Index 
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = FAISS.from_documents(documents = splits,
                                    embedding = hf_embeddings)

retriever = vectorstore.as_retriever()                                

## Multi Query Prompt

In [40]:
from langchain.prompts import ChatPromptTemplate


# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatGroq(temperature = 0)
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)

In [41]:
generate_queries.invoke("What is Task Decomposition")

['1. Can you explain the concept of task decomposition in artificial intelligence?',
 '2. How does task decomposition work in the field of computer science?',
 '3. What are the different methods used for task decomposition?',
 '4. Can you provide examples of tasks that can be decomposed?',
 '5. What is the significance of task decomposition in problem-solving processes?']

In [42]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    ## Get unique Documents
    unique_docs = list(set(flattened_docs))

    ## Return 
    return [loads(doc) for doc in unique_docs]


## Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

10

In [43]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\nAnother quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-specific PDDL and a suitable planner

In [44]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


## RAG 
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM (Large Language Model) agents is the process of breaking down large tasks into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently. Task decomposition can be done using simple prompting techniques, task-specific instructions, or with human inputs. For example, the agent can be prompted with "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?" to decompose a task. Alternatively, task-specific instructions such as "Write a story outline." can be used for specific tasks. Human inputs can also be used for task decomposition.\n\nAnother approach to task decomposition for LLM agents is LLM+P, which involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. The LLM translates the problem into "Problem PDDL", requests a classical planner to generate a PDDL pl

## Part 6: RAG-Fusion

Flow: 

![](/Users/dhavalantala/Desktop/langchain_/Photo/RAG_fusion.png)

### Prompt

In [45]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

generate_queries = (
    prompt_rag_fusion
    | ChatGroq(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [47]:
generate_queries

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are a helpful assistant that generates multiple search queries based on a single input query. \n\nGenerate multiple search queries related to: {question} \n\nOutput (4 queries):'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x16832d1c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x16832d3a0>, temperature=1e-08, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(...)

## RRF Explanation

#### Question Rankings

- **Question A**: 
  1. Doc1
  2. Doc4
  3. Doc3
  4. Doc2

- **Question B**: 
  1. Doc3
  2. Doc1
  3. Doc2
  4. Doc4

- **Question C**: 
  1. Doc4
  2. Doc3
  3. Doc1
  4. Doc2

### Rank Positions

- **Doc1**:
  - Question A rank: 1
  - Question B rank: 2
  - Question C rank: 3

- **Doc2**:
  - Question A rank: 4
  - Question B rank: 3
  - Question C rank: 4

- **Doc3**:
  - Question A rank: 3
  - Question B rank: 1
  - Question C rank: 2

- **Doc4**:
  - Question A rank: 2
  - Question B rank: 4
  - Question C rank: 1

### Reciprocal Rank Fusion Calculation

Using `k = 60`:

#### Doc1
- Reciprocal Rank (Question A): `1 / (60 + 1) = 1 / 61`
- Reciprocal Rank (Question B): `1 / (60 + 2) = 1 / 62`
- Reciprocal Rank (Question C): `1 / (60 + 3) = 1 / 63`
- **RRF(Doc1)**: `1 / 61 + 1 / 62 + 1 / 63 ≈ 0.0487`

#### Doc2
- Reciprocal Rank (Question A): `1 / (60 + 4) = 1 / 64`
- Reciprocal Rank (Question B): `1 / (60 + 3) = 1 / 63`
- Reciprocal Rank (Question C): `1 / (60 + 4) = 1 / 64`
- **RRF(Doc2)**: `1 / 64 + 1 / 63 + 1 / 64 ≈ 0.0469`

#### Doc3
- Reciprocal Rank (Question A): `1 / (60 + 3) = 1 / 63`
- Reciprocal Rank (Question B): `1 / (60 + 1) = 1 / 61`
- Reciprocal Rank (Question C): `1 / (60 + 2) = 1 / 62`
- **RRF(Doc3)**: `1 / 63 + 1 / 61 + 1 / 62 ≈ 0.0487`

#### Doc4
- Reciprocal Rank (Question A): `1 / (60 + 2) = 1 / 62`
- Reciprocal Rank (Question B): `1 / (60 + 4) = 1 / 64`
- Reciprocal Rank (Question C): `1 / (60 + 1) = 1 / 61`
- **RRF(Doc4)**: `1 / 62 + 1 / 64 + 1 / 61 ≈ 0.0484`

### Final Ranking

Based on the RRF scores:

1. **Doc1**: `≈ 0.0487`
2. **Doc3**: `≈ 0.0487`
3. **Doc4**: `≈ 0.0484`
4. **Doc2**: `≈ 0.0469`

In [48]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)


6

In [49]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down a complex task into smaller, manageable subgoals. This can be done in several ways, such as using simple prompting techniques like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?", using task-specific instructions, or with human inputs. Another approach, called LLM+P, involves relying on an external classical planner to do long-horizon planning using the Planning Domain Definition Language (PDDL) as an intermediate interface. This approach outsources the planning step to an external tool, assuming the availability of domain-specific PDDL and a suitable planner. Self-reflection is also an important aspect of task decomposition for autonomous agents, as it allows them to improve iteratively by refining past action decisions and correcting previous mistakes.'

## Part 7: Decomposition

In [56]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [57]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGroq(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [58]:
questions

['1. "Main components of a Language Model-based autonomous agent system"',
 '2. "Role of large language models in autonomous agent systems"',
 '3. "How are perception, decision-making, and action implemented in LLM-powered autonomous agents?"']

### Answer recursively


![](/Users/dhavalantala/Desktop/langchain_/Photo/answer_recursively.png)

In [59]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [60]:
decomposition_prompt

ChatPromptTemplate(input_variables=['context', 'q_a_pairs', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'q_a_pairs', 'question'], template='Here is the question you need to answer:\n\n\n --- \n {question} \n --- \n\n\nHere is any available background question + answer pairs:\n\n\n --- \n {q_a_pairs} \n --- \n\n\nHere is additional context relevant to the question: \n\n\n --- \n {context} \n --- \n\n\nUse the above context and any background question + answer pairs to answer the question: \n {question}\n'))])

In [61]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatGroq(temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [62]:
q_a_pairs

'\n---\nQuestion: 1. "Main components of a Language Model-based autonomous agent system"\nAnswer: Based on the provided document and additional context, the main components of a Language Model-based autonomous agent system can be summarized as follows:\n\n1. **Language Model**: At the core of the system is a large language model, which is responsible for processing natural language inputs and generating appropriate responses. Recent advancements in language models, such as Chain of Thought prompting (CoT) and Tree of Thoughts, have further enhanced their ability to reason and solve problems.\n\n2. **Knowledge Sources**: External knowledge sources, such as databases, APIs, or web pages, can be integrated into the system to provide additional information and context for the language model. This can help the agent to make more informed decisions and generate more accurate responses.\n\n3. **Discrete Reasoning**: A discrete reasoning module can be added to the system to enable symbolic man

In [63]:
answer

"In LLM-powered autonomous agents, perception, decision-making, and action are implemented through a combination of the large language model (LLM) and other key components.\n\n1. **Perception**: Perception in these agents is primarily handled by the LLM, which processes natural language inputs and generates responses. Recent advancements in language models, such as Chain of Thought prompting (CoT) and Tree of Thoughts, have further enhanced their ability to reason and solve problems based on the input data. Additionally, external knowledge sources, such as databases, APIs, or web pages, can be integrated into the system to provide additional information and context for the language model, improving its perception capabilities.\n\n2. **Decision-Making**: Decision-making in LLM-powered autonomous agents is implemented through a combination of the LLM and other components. The LLM is responsible for processing input data, generating potential actions, and evaluating their outcomes. Compon

## Answer individually

![](/Users/dhavalantala/Desktop/langchain_/Photo/answer_individually.png)

In [64]:
## Answer each sub-question individually

from langchain import hub 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

## Rag prompt 
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/var/folders/m8/xhbf16sj00b_px7r2lbh9dlr0000gn/T/ipykernel_19473/3746895329.py:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [66]:
questions

['1. "Main components of a Language Model-based autonomous agent system"',
 '2. "Role of large language models in autonomous agent systems"',
 '3. "How are perception, decision-making, and action implemented in LLM-powered autonomous agents?"']

In [65]:
answers

['The main components of a Language Model-based autonomous agent system include large language models, external knowledge sources, discrete reasoning, tools, and dynamic memory. These components enable the agent to perform tasks such as question-answering, problem-solving, and interactive simulations of human behavior.',
 'Large language models (LLMs) play a significant role in autonomous agent systems. They can be empowered with optimal planning proficiency, aligned with feedback, and used for reasoning and problem-solving tasks. Additionally, they can be integrated with various tools and external knowledge sources to enhance their capabilities.',
 "Perception, decision-making, and action in LLM-powered autonomous agents are implemented through a combination of the large language model (LLM) and additional components. The LLM functions as the agent's brain, handling planning, subgoal decomposition, reflection, and refinement. Planning involves task decomposition, which can be achieved

In [67]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"The main components of an LLM-powered autonomous agent system include:\n\n1. Large Language Models (LLMs): These models serve as the core of the autonomous agent system, handling planning, subgoal decomposition, reflection, and refinement. They can be empowered with optimal planning proficiency, aligned with feedback, and used for reasoning and problem-solving tasks.\n\n2. External Knowledge Sources: These are integrated with the LLMs to enhance their capabilities. They provide the agent with additional information and data that can be used for question-answering, problem-solving, and interactive simulations of human behavior.\n\n3. Discrete Reasoning: This component enables the agent to make logical and rational decisions based on the information and data provided by the LLMs and external knowledge sources.\n\n4. Tools: These are various tools that the agent can use to perform tasks. They can be integrated with the LLMs to enable the agent to perform a wide range of functions.\n\n5. 

## Part 8: Step Back

![](/Users/dhavalantala/Desktop/langchain_/Photo/step_back.png)

In [68]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [69]:
generate_queries_step_back = prompt | ChatGroq(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'how do LLM agents break down tasks? or what is the process of dividing tasks for LLM agents? or how are tasks handled by LLM agents?'

In [70]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGroq(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down a complex task into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently. Task decomposition can be done in several ways:\n\n1. By LLM with simple prompting: The LLM is prompted with questions like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?" to decompose the task.\n2. By using task-specific instructions: For example, "Write a story outline." for writing a novel.\n3. With human inputs: Human inputs can also be used to decompose a task.\n\nAnother approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. In this approach, the LLM translates the problem into "Problem PDDL", requests a classical planner to generate a PDDL plan based on an existing "Domain PDDL", and then translates the PDDL plan back into natural language. This approach outsources the planning step to an ext

## Part 9: HyDE

![](/Users/dhavalantala/Desktop/langchain_/Photo/HyDE.png)

In [71]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_docs_for_retrieval = (
    prompt_hyde | ChatGroq(temperature=0) | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition is a crucial aspect of building Large Language Model (LLM) agents that can effectively complete complex tasks. It involves breaking down a complex task into a series of smaller, manageable subtasks that the LLM agent can handle more efficiently. This process is similar to the way humans approach complex problems by dividing them into smaller components and tackling each one step by step.\n\nIn the context of LLM agents, task decomposition can be used to simplify a wide range of complex tasks, such as text summarization, question answering, and machine translation. For example, in text summarization, a complex task might be broken down into several subtasks, such as identifying the most important sentences, extracting key phrases, and generating a concise summary.\n\nTask decomposition has several benefits for LLM agents. First, it allows the agent to focus on one subtask at a time, which can help to reduce errors and improve the overall quality of the output. Second

In [72]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [73]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down large tasks into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently. Task decomposition can be done in several ways for LLM agents, such as using simple prompting techniques, task-specific instructions, or human inputs. Additionally, there is an approach called LLM+P (Liu et al. 2023) that involves relying on an external classical planner for long-horizon planning using the Planning Domain Definition Language (PDDL) as an intermediate interface. However, LLMs face challenges in long-term planning and task decomposition, such as adjusting plans when faced with unexpected errors and the restricted context capacity that limits the inclusion of historical information. The reliability of the natural language interface between LLMs and external components is also a challenge, as LLMs may make formatting errors and occasionally exhibit rebellious behavior.